# Assignment11
* 2019220177 Seungwook

In [5]:
%cd /content/drive/My\ Drive/MIMP

/content/drive/My Drive/MIMP


## Initial Setting

### git Setting

In [6]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Assignment10/Assignment10.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Assignment11/

no changes added to commit (use "git add" and/or "git commit -a")


In [9]:
!git config --global user.email "sooonchang@gmail.com"
!git config --global user.name "SooonChang"

In [ ]:
!git 

## 0. Optimization

### Import libraries

In [ ]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
